In [ ]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pyperclip
import pyautogui
import time

driver = Driver(uc=True)
url = "https://www.imovelweb.com.br/apartamentos-aluguel-jardim-cidade-universitaria-castelo-branco-joao-pessoa-portal-do-sol-joao-pessoa.html"
driver.uc_open_with_reconnect(url, 4)
driver.uc_gui_click_captcha()

imovel = []
titulos = []
subtitulos = []
vendas = []
condominioIptus = []
enderecos = []
iconFeatures = []
descricoes = []
caracteristicas = []
fotos = []
dados = []

anuncios = driver.find_elements(By.CLASS_NAME, "postingCard-module__posting-description")

anuncios[0].click()
    
titulos.append(driver.find_element(By.CLASS_NAME, "title-property").text)
subtitulos.append(driver.find_element(By.CLASS_NAME, "title-type-sup-property").text)
vendas.append(driver.find_element(By.CLASS_NAME, "price-value").text)
condominioIptus.append(driver.find_element(By.CLASS_NAME, "price-expenses").text)
enderecos.append(driver.find_element(By.CLASS_NAME, "section-location-property").text)
descricoes.append(driver.find_element(By.CLASS_NAME, "description-module__wrapper-description___2rEoY").text)

buttons = int(driver.find_element(By.ID, "new-gallery-portal").text)
pyautogui.moveTo(394, 619)  
pyautogui.click()

for i in range(buttons):
    pyautogui.moveTo(818, 664)  
    pyautogui.rightClick()
    pyautogui.moveTo(917, 798)
    pyautogui.click()

    image_src = pyperclip.paste()
    fotos.append(image_src)
    
    pyautogui.moveTo(1585, 636)
    pyautogui.click()
    time.sleep(3)

driver.close()


In [17]:
def adicionarLinhas():
    print(116*"-")

def linhaEndBeg():
    print(100*"=")

for i in range(len(titulos)):
    adicionarLinhas()
    print(f"Título: {titulos[i]}")
    adicionarLinhas()
    print(f"Subtítulo: {subtitulos[i]}")
    adicionarLinhas()
    print(f"Preço de Venda: {vendas[i]}")
    adicionarLinhas()
    print(f"Condomínio/IPTU: {condominioIptus[i]}")
    adicionarLinhas()
    print(f"Endereço: {enderecos[i]}")
    adicionarLinhas()
    descricoes[i] = descricoes[i].replace("\n", " ")
    print(f"Descrição:\n{descricoes[i] if descricoes[i] else 'Sem descrição disponível'}")
    adicionarLinhas()
    
    print("Fotos:")
    for foto in fotos:
        print(f"   {foto}")
    
    adicionarLinhas()
    print("\n")


--------------------------------------------------------------------------------------------------------------------
Título: Happy Manaíra, disponível para venda, na Praia de Manaíra, João Pessoa - PB
--------------------------------------------------------------------------------------------------------------------
Subtítulo: Apartamento · 34m² · 1 quarto · 1 vaga
--------------------------------------------------------------------------------------------------------------------
Preço de Venda: venda R$ 450.000
--------------------------------------------------------------------------------------------------------------------
Condomínio/IPTU: Condomínio R$ 600 · IPTU R$ 43
--------------------------------------------------------------------------------------------------------------------
Endereço: AV João Maurício, 1185, Manaíra, João Pessoa
--------------------------------------------------------------------------------------------------------------------
Descrição:
Happy Manaíra  Co

In [18]:
import time
from fpdf import FPDF
import re

# Criar uma instância do FPDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Definir margens e fontes
pdf.set_left_margin(15)
pdf.set_right_margin(15)

# Fonte para título
pdf.set_font("Arial", 'B', 16)
pdf.cell(200, 10, txt="Informações do Imóvel", ln=True, align='C')
pdf.ln(10)  # Adiciona espaço depois do título

# Função para remover caracteres especiais e emojis
def clean_content(content):
    # Substituir emojis e caracteres especiais não compatíveis com latin-1
    content = content.replace("\u2013", "-")  # Substitui n-dash por hífen simples
    content = content.replace("\u2014", "-")  # Substitui em dash por hífen
    content = content.replace("\u2022", "-")  # Substitui bullet por hífen
    # Remover qualquer emoji ou caractere especial usando regex
    content = re.sub(r'[^\x00-\x7FáéíóúãõâêîôûàèìòùçÁÉÍÓÚÃÕÂÊÎÔÛÀÈÌÒÙÇ]+', '', content)
    
    content = content.replace("\n", " ")  # Substitui todas as quebras de linha por espaço
    return content

# Função para adicionar título e conteúdo com formatação
def add_section(pdf, title, content):
    # Limpar o conteúdo para remover caracteres não compatíveis
    content = clean_content(content)

    # Adiciona o título da seção (em negrito)
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 8, title + ":", ln=True)

    # Adiciona o conteúdo da seção
    pdf.set_font("Arial", '', 10)  # Reduzindo a fonte do conteúdo

    # Identificar e tornar links clicáveis
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', content)
    cursor_position = 0  # Marca a posição do texto

    for url in urls:
        start_pos = content.find(url, cursor_position)
        if start_pos != -1:
            # Adiciona o texto até o link
            pdf.multi_cell(0, 6, content[cursor_position:start_pos])
            cursor_position = start_pos + len(url)

            # Adiciona o link clicável
            pdf.set_text_color(0, 0, 255)  # Definir a cor azul para o link
            pdf.cell(0, 6, url, link=url)  # Cria um link clicável
            pdf.set_text_color(0, 0, 0)  # Voltar à cor padrão

    # Adiciona o restante do conteúdo após o último link
    pdf.multi_cell(0, 6, content[cursor_position:])
    
    # Linha separadora
    pdf.cell(0, 5, '-' * 150, ln=True)  # Reduzindo a altura da linha separadora
    pdf.ln(3)  # Ajustando o espaçamento após a linha separadora


# Adicionar as informações de cada imóvel
for i in range(len(titulos)):
    # Título
    add_section(pdf, "Título", titulos[i])

    # Subtítulo
    add_section(pdf, "Subtítulo", subtitulos[i])

    # Preço de venda
    add_section(pdf, "Preço de Venda", vendas[i])

    # Condomínio/IPTU
    add_section(pdf, "Condomínio/IPTU", condominioIptus[i])

    # Endereço
    add_section(pdf, "Endereço", enderecos[i])

    # Descrição
    add_section(pdf, "Descrição", descricoes[i] if descricoes[i] else 'Sem descrição disponível')

    # Fotos
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(0, 10, "Fotos:", ln=True)
    pdf.set_font("Arial", '', 10)
    for foto in fotos:
        pdf.multi_cell(0, 5, f"- {foto}")
    pdf.ln(1)
pdf.cell(0, 5, '-' * 150, ln=True)  # Reduzindo a altura da linha separadora

# Gerar nome do arquivo com timestamp
timestamp = time.strftime("%Y%m%d_%H%M%S")
filename = f"informacoes_imovel_{timestamp}.pdf"

# Definir um caminho absoluto ou relativo
output_path = f"./PDF_files/{filename}"  # Salva no diretório atual

# Salvar o PDF com o novo nome
pdf.output(output_path)

print(f"PDF gerado com sucesso! Nome do arquivo: {output_path}")


PDF gerado com sucesso! Nome do arquivo: ./PDF_files/informacoes_imovel_20250327_122814.pdf
